## 모델 학습

In [1]:
!pip install ultralytics
!pip install bytetracker
!pip install supervision==0.1.0

In [2]:
import cv2
import os
from ultralytics import YOLO

# 모델 초기화
model = YOLO('yolov8m.pt')

# 데이터셋 경로 지정
data_path = '/home/piai/posco/ai/project/modeling/dataset/data.yaml'  # YAML 파일경로지정하기(pc에있는거)

# 모델 학습
model.train(data=data_path, epochs=35, imgsz=640, batch=16) # 에폭수 이런건 나중에 내가조절할 것

## zone counting

In [6]:
from ultralytics import YOLO

# train63,,이런거 숫자 바꿔줄것, path경로는 주피터에 있는걸로 사용
model = YOLO('best.pt')

import cv2

# 인덱스가 1갠데 이상한거 써서 오류났던거, 해결완료
car_class_index = 0

def filter_boxes(boxes, names):
    filtered_boxes = []
    for box_data in boxes.data:
        x1, y1, x2, y2, conf, cls_idx = box_data
        if cls_idx.item() == car_class_index:  # Filter only for cars
            cls_name = names[int(cls_idx.item())]
            filtered_boxes.append((cls_name, conf.item(), [x1.item(), y1.item(), x2.item(), y2.item()]))
    return filtered_boxes

def count_zone(bbox, frame_width, frame_height):
    zones = [
        (0, 0, frame_width//3, frame_height//3),
        (frame_width//3, 0, 2*frame_width//3, frame_height//3),
        (2*frame_width//3, 0, frame_width, frame_height//3),
        (0, frame_height//3, frame_width//3, 2*frame_height//3),
        (frame_width//3, frame_height//3, 2*frame_width//3, 2*frame_height//3),
        (2*frame_width//3, frame_height//3, frame_width, 2*frame_height//3),
        (0, 2*frame_height//3, frame_width//3, frame_height),
        (frame_width//3, 2*frame_height//3, 2*frame_width//3, frame_height),
        (2*frame_width//3, 2*frame_height//3, frame_width, frame_height)
    ]
    zone_counts = [0] * len(zones)
    for i, zone in enumerate(zones):
        if bbox[0] < zone[2] and bbox[2] > zone[0] and bbox[1] < zone[3] and bbox[3] > zone[1]:
            zone_counts[i] += 1
    return zone_counts

def draw_zones(annotated_frame, frame_width, frame_height):
    # Draw lines for each zone with a more visible color and thicker lines
    # Vertical lines
    cv2.line(annotated_frame, (frame_width//3, 0), (frame_width//3, frame_height), (0, 255, 255), 4)
    cv2.line(annotated_frame, (2*frame_width//3, 0), (2*frame_width//3, frame_height), (0, 255, 255), 4)
    # Horizontal lines
    cv2.line(annotated_frame, (0, frame_height//3), (frame_width, frame_height//3), (0, 255, 255), 4)
    cv2.line(annotated_frame, (0, 2*frame_height//3), (frame_width, 2*frame_height//3), (0, 255, 255), 4)

cap = cv2.VideoCapture(0)

while cap.isOpened():

    if success:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(frame_rgb)
        
        if len(results) > 0:
            filtered_results = filter_boxes(results[0].boxes, results[0].names)
            
            total_zone_counts = [0] * 9
            
            annotated_frame = frame.copy()
            for name, conf, bbox in filtered_results:
                bbox = [int(coord) for coord in bbox]
                
                zone_counts = count_zone(bbox, frame.shape[1], frame.shape[0])
                total_zone_counts = [sum(x) for x in zip(total_zone_counts, zone_counts)]
                
                cv2.rectangle(annotated_frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
                cv2.putText(annotated_frame, f'{name} {conf:.2f}', (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Draw zones on the annotated frame
            draw_zones(annotated_frame, frame.shape[1], frame.shape[0])
            
            for i, count in enumerate(total_zone_counts):
                cv2.putText(annotated_frame, f'Zone {i+1}: {count}', (10, 20*(i+1)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

            cv2.imshow("Filtered YOLOv8 Inference", annotated_frame)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
    else:
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 288.6ms
Speed: 3.0ms preprocess, 288.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 311.3ms
Speed: 1.2ms preprocess, 311.3ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 248.4ms
Speed: 1.1ms preprocess, 248.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 254.3ms
Speed: 1.0ms preprocess, 254.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 259.4ms
Speed: 2.2ms preprocess, 259.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 264.9ms
Speed: 1.1ms preprocess, 264.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 243.5ms
Speed: 2.1ms preprocess, 243.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 286.0ms
Speed: 3.1ms prepr

## in/out counting

In [1]:
import cv2
from ultralytics import YOLO
from ultralytics.solutions import object_counter

# YOLO 모델 로드
model = YOLO("best.pt")

# 비디오 파일 열기
video_path = "final_1.mov"
cap = cv2.VideoCapture(video_path)



# 라벨 위치 정의
labels_positions = [
    ((600, 50), "Line 1"),  # Line 1 레이블 위치
    ((600, 550), "Line 2"),  # Line 2 레이블 위치
    ((290, 300), "Line 3"),  # Line 3 레이블 위치
    ((900, 300), "Line 4")   # Line 4 레이블 위치
]

# 중앙 정사각형 좌표 정의
center_x, center_y = 600, 400  # 1920x1080 해상도의 중앙
half_size = 200  # 정사각형 반 변의 길이 (총 200 픽셀)

# 카운팅할 라인 포인트 (정사각형 모양)
line_points = [
    [(center_x - half_size, center_y - half_size), (center_x + half_size, center_y - half_size)],  # 첫 번째 카운팅 라인 (상단 가로)
    [(center_x - half_size, center_y + half_size), (center_x + half_size, center_y + half_size)],  # 두 번째 카운팅 라인 (하단 가로)
    [(center_x - half_size, center_y - half_size), (center_x - half_size, center_y + half_size)],  # 세 번째 카운팅 라인 (좌측 세로)
    [(center_x + half_size, center_y - half_size), (center_x + half_size, center_y + half_size)]   # 네 번째 카운팅 라인 (우측 세로)
]

classes_to_count = [0]  # 카운팅할 클래스 인덱스 (예: 차량)

# 라벨 위치 정의
labels_positions = [
    ((570, 170), "Line 1"),  # Line 1 레이블 위치
    ((570,630), "Line 2"),  # Line 2 레이블 위치
    ((290, 312), "Line 3"),  # Line 3 레이블 위치
    ((820, 312), "Line 4")   # Line 4 레이블 위치
]

# ObjectCounter 객체 초기화
counters = [object_counter.ObjectCounter() for _ in range(4)]
for counter, pts in zip(counters, line_points):
    counter.set_args(view_img=False,
                     reg_pts=pts,
                     classes_names=model.names,
                     draw_tracks=False,
                     line_thickness=2)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("웹캠에서 프레임을 가져오는 데 실패했습니다.")
        break
    
    # 객체 추적 및 특정 클래스 카운트
    tracks = model.track(im0, persist=True, show=False, classes=classes_to_count)

    # 객체 카운팅 프로세스 시작 및 결과 출력
    print("각 라인별 합계:")
    for i, (counter, (pos, label)) in enumerate(zip(counters, labels_positions)):
        im0 = counter.start_counting(im0, tracks)
        if i in [0, 2]:  # Line 1과 Line 3은 out만
            total_counts = counter.out_counts
        else:            # Line 2와 Line 4는 in만
            total_counts = counter.in_counts
        
        # 레이블 위치 조정 및 텍스트 출력
        pos = (int(pos[0]), int(pos[1]))
        cv2.putText(im0, f"Count: {total_counts}", pos, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        print(f"{label}: {total_counts}")

    cv2.imshow("Object Counting", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 정리
cap.release()
cv2.destroyAllWindows()

Line Counter Initiated.
Line Counter Initiated.
Line Counter Initiated.
Line Counter Initiated.
